# Objective

Here we will scrape infos of dates do `Availability` and `Offered Since` of each new page in our database.

# Libs

In [1]:
# Import Libs
from bs4 import BeautifulSoup
from datetime import date
from datetime import datetime, timedelta
from folium.plugins import FastMarkerCluster
from requests import get
import datetime
import datetime as dt
import folium
import pandas as pd
import streamlit as st
today = date.today()

# Imports

In [2]:
df_pararius = pd.read_csv('app/df_coo_pararius.csv',index_col=[0])
house_temp  = pd.read_csv('data/processed/house_temp.csv',index_col=[0])

# What is new?
Here we will scrape infos of dates do `Availability` and `Offered Since` of each new page in our database.

In [3]:
temp1 = df_pararius['url'].unique()
temp2 = house_temp['url'].unique()
urls = list(set(temp1) - set(temp2))
print(len(urls))

298


# Get data from table

Going in each page, I want to scrape tables to get specific dates

In [4]:
def key_val(text):
    part = html_soup.find("dt",text=text)
    key = (part.text.strip())
    val = (part.findNext("dd").text.strip())
    return key, val

Getting data and saving in a variable

In [5]:
not_available=[]
description_2=[]

for url in urls:
    
    response = get(url)
    description_1={}
    html_soup = BeautifulSoup(response.text,'html.parser')
    
    
    try:
        key1, val1 = key_val("Offered since")
        key2, val2 = key_val("Available")
        description_1.update({
            'url':url,
            key1:val1,
            key2:val2,
        })
        description_2.append(description_1)
        
    except:
        pass

# Add dates into original dataframe

In [6]:
house_temp_ = pd.DataFrame(description_2)
house_temp = pd.concat([house_temp, house_temp_],0)

<ipython-input-6-e6a6dab0263b>:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  house_temp = pd.concat([house_temp, house_temp_],0)


# Transform date str into Date Time

function to transform data

In [7]:
def fixing_time_delta(df, time, my_ofset):
    fixing_time_delta = df.loc[df['Offered since'].str.contains(time, na=False), 'Offered since']
    fixing_time_delta = fixing_time_delta.str.replace("\D","",regex=True).astype(int)
    fixing_time_delta = today - fixing_time_delta.apply(my_ofset)
    fixing_time_delta = pd.to_datetime(fixing_time_delta).dt.strftime('%d-%m-%Y')
    df.loc[df['Offered since'].str.contains(time, na=False), 'Offered since'] = fixing_time_delta
    return df

appling function

In [8]:
try:
    house_temp = fixing_time_delta(house_temp,"week",pd.offsets.Week)
    house_temp = fixing_time_delta(house_temp,"month",pd.offsets.MonthBegin)
except:
    print('erro')
    pass

In [9]:
today = date.today()
house_temp['Available'] = house_temp['Available'].str.replace("From","")
house_temp['Available'] = house_temp['Available'].str.replace("Immediately|In consultation",str(today))
house_temp['Available'] = pd.to_datetime(house_temp['Available']).dt.strftime('%d-%m-%Y')

<ipython-input-9-ffea408b42f7>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  house_temp['Available'] = house_temp['Available'].str.replace("Immediately|In consultation",str(today))


# Save house_temp

In [10]:
house_temp = house_temp.drop_duplicates(subset=['url']).reset_index(drop=True)

In [11]:
house_temp.to_csv('data/processed/house_temp.csv')

# Add house temp into df_pararius

In [12]:
result = pd.merge(df_pararius, house_temp, how='left', on='url')

In [13]:
#result['Offered since'] = pd.to_datetime(result['Offered since'], format='%d-%m-%Y')

# Save whole DataFrame

In [14]:
result2 = result[[
    'deal',
    'garden-surface-area',
    'img',
    'price',
    'link',
    'agency',
    'surface-area',
    'interior',
    'latitude',
    'longitude',
    'number-of-rooms',
    'Plaats',
    'url',
    'date',
    'status',
    'irl',
    'image',
    'address',
    'street',
    'Offered since',
    'Available',
]]

In [15]:
result2 = result2.rename(columns={
    'img': 'Add',
    'price': 'Price',
    'Plaats': 'City',
    'surface-area': 'Area',
    'number-of-rooms': 'Rooms',
    'garden-surface-area': 'Garden',
    'Offered since': 'Offered',
    'Available': 'Available',})

In [16]:
result2.to_csv('app/df_coo_pararius2.csv')

In [ ]:
qwaszxqwaszx

In [23]:
result2[result2['Available']=='28-03-2024']

,deal,Garden,Add,Price,link,agency,Area,interior,latitude,longitude,...,City,url,date,status,irl,image,address,street,Offered,Available
4989,0.049211,0.0,<a href=https://www.pararius.com/apartment-for...,1585.0,<a target='_blank' href=https://www.pararius.c...,Rotterdam Apartments,76,Furnished,51.929868,4.489119,...,Rotterdam,https://www.pararius.com/apartment-for-rent/ro...,2021-10-19,Free,/apartment-for-rent/rotterdam/15ca8588/crooswi...,https://casco-media-prod.global.ssl.fastly.net...,3034 CH Rotterdam (Oud Crooswijk),Apartment Crooswijksesingel,01-08-2021,28-03-2024
